# Checking enviroment states and action sets

In [1]:
import sys
sys.path.append("..")
from SARSA_Agent import *
from ENV_DETAILS import *
from RUN_TENSORBOARD import *

events_folder = "./logs_hyper"
main("./logs_hyper")

2024-03-03 07:51:05.529929: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Selected port: 55851


2024-03-03 07:51:06.895737: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:55851/ (Press CTRL+C to quit)


In [2]:

ENV = "LunarLander-v2"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:
env = gym.make(ENV)#,new_step_api=True
env

<TimeLimit<OrderEnforcing<LunarLander<LunarLander-v2>>>>

In [4]:
s = env.observation_space.sample()
s, s.shape

(array([-0.05034237, -0.42907217,  0.25956962,  0.8996386 ,  0.17745754,
        -1.0393299 ,  0.63083696, -0.5282965 ], dtype=float32),
 (8,))

In [5]:
env.observation_space, env.observation_space.shape

(Box(-inf, inf, (8,), float32), (8,))

In [6]:
s = env.reset()
env.action_space.n, env.action_space, env.step(1)

(4,
 Discrete(4),
 (array([ 0.01491423,  1.4020689 ,  0.74720865, -0.20954564, -0.01482209,
         -0.12267981,  0.        ,  0.        ], dtype=float32),
  0.4194193755830906,
  False,
  {}))

### Training/Hyperparam run

In [7]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_SARSA/"

In [8]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_SARSA"
    for exploration_tech in ["epsilon"]:#, "boltzman"]:

        project_name = "keras_tunning_"+exploration_tech
        tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/SARSA_"+exploration_tech+"/", exploration_tech =exploration_tech,
                         sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value = SUCESS_CRITERIA_VALUE,
                  end_of_episode = EPISODES, batch = 32,  evaluation_epoch = 2000, 
                   training_steps = 700000, 
                   time_to_update_min = 200, time_to_update_max = 800,
                   lr_min = 0.000005, lr_max = 0.005,
                   discount_min = 0.90, discount_max = 0.99,
                   dense_min = 32, dense_max = 256,
                   environment_name = ENV,
                   max_n_layers = 2),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            # distribution_strategy=tf.distribute.MirroredStrategy(),
            directory= dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])


else : 
    print("Acquiring parameters ....")
    learning_rate = 0.1
    training_steps = 5000000
    exploration_tech = "epsilon"
    discount = 0.99
    time_to_update = 100
    max_n_layers = 1
    dense_units = [32]
    end_of_episode = EPISODES
    sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH
    sucess_criteria_value = SUCESS_CRITERIA_VALUE
    run_training(training_steps, learning_rate,  exploration_tech, discount, time_to_update, max_n_layers, dense_units, writer, end_of_episode, sucess_criteria_epochs, sucess_criteria_value)

Trial number :  1

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
0.97              |0.97              |discount
0.001672          |0.001672          |learning_rate
800               |800               |time_to_update
1                 |1                 |n_dense_layers
180               |180               |dense_units_0

Trial number :  1


In [ ]:
val_env = gym.make(ENV)

for exploration_tech in ["epsilon", "boltzman"]:

    project_name = "keras_tunning_"+exploration_tech
    tuner = kt.BayesianOptimization(
        MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/SARSA_"+exploration_tech+"/", exploration_tech =exploration_tech,
                        sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value = SUCESS_CRITERIA_VALUE,
                end_of_episode = EPISODES, batch = 32,  evaluation_epoch = 2000, 
                training_steps = 700000, 
                time_to_update_min = 200, time_to_update_max = 800,
                lr_min = 0.000005, lr_max = 0.005,
                discount_min = 0.90, discount_max = 0.99,
                dense_min = 32, dense_max = 256,
                environment_name = ENV,
                max_n_layers = 2),
        objective= kt.Objective('total_train_reward', direction="max"), 
        max_trials = 50,
        directory= dir,
        project_name=project_name
    )

    for trials in tuner.oracle.get_best_trials(num_trials=3):
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
        

    print(tuner.get_best_models())
    env_model = tuner.get_best_models()[0]
    final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./SARSA_"+exploration_tech+"_video.mp4")
    print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

    